<a href="https://colab.research.google.com/github/niit-ibm/lt4-pe-lab2/blob/main/pe_lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulation 2: Creating Reusable Prompt Templates
## Hands-On Lab Using IBM Granite Model via Replicate

### Scenario
You are a Business Analyst in a growing consulting firm. Your team works across multiple client accounts, and each week several analysts produce project status updates, extract insights from meeting transcripts, draft client-ready emails, and respond to recurring queries.

Although everyone uses IBM Watsonx Prompt Lab, the prompts written across the team vary widely in structure, tone, and level of detail. As a result, the outputs are inconsistent and require significant manual clean-up before they can be shared with clients.

### Challenge
Your manager wants AI-generated content to follow a consistent, repeatable standard across the team. You've been asked to create the first set of **reusable prompt templates** that anyone can use to produce high-quality, uniform outputs by simply updating a few key variables.

### Learning Objective
Create prompt templates with variables to generate consistent, scalable outputs from LLMs.

## Setup: Install Required Libraries

First, install the necessary libraries as used in the reference implementation.

In [ ]:
# Install required libraries
!pip install "langchain_community<0.3.0" replicate ipywidgets ibm-granite-community --quiet

## Configure Environment and Import Libraries

In [ ]:
import os
import replicate
from langchain_community.llms import Replicate
from ibm_granite_community.notebook_utils import get_env_var
import ipywidgets as widgets
from IPython.display import display, Markdown, HTML
from string import Template

## Set Replicate API Token

You need a Replicate API token to use the IBM Granite model. Get your token from [replicate.com](https://replicate.com/account/api-tokens)

In [ ]:
# Set your Replicate API token
# Option 1: Set it directly (not recommended for production)
# os.environ["REPLICATE_API_TOKEN"] = "your-token-here"

# Option 2: Use the utility function to get from environment or prompt
replicate_api_token = get_env_var("REPLICATE_API_TOKEN")
os.environ["REPLICATE_API_TOKEN"] = replicate_api_token

## Initialize the IBM Granite Foundation Model

We'll use the **IBM Granite 3.3 8B Instruct** model via Replicate.

In [ ]:
# Model configuration - matching reference implementation
MODEL_NAME = "ibm-granite/granite-3.3-8b-instruct"
MAX_TOKENS = 1024
TEMPERATURE = 0.2

# Initialize the model
llm = Replicate(
    model=MODEL_NAME,
    model_kwargs={
        "max_tokens": MAX_TOKENS,
        "temperature": TEMPERATURE
    }
)

print(f"✅ Model initialized: {MODEL_NAME}")
print(f"   Max Tokens: {MAX_TOKENS}")
print(f"   Temperature: {TEMPERATURE}")

---
## Task 1: Draft a Baseline Prompt for One Recurring Task

### Teaching Moment
Before creating a template, you need to draft a detailed, working prompt for a specific use case. This helps you understand what information is needed and what should become variables.

### Common Recurring Task
Let's focus on **writing weekly status summaries** for client projects - a task nearly all analysts perform.

In [ ]:
# Task 1: Draft an initial prompt for a specific project
# This is NOT a template yet - it's hard-coded for one specific use case

baseline_prompt = """Write a concise weekly status summary for the DeltaFin client project.
Highlight progress made this week, note any blockers, and outline next steps in a professional
tone suitable for a client email."""

print("🔵 TASK 1: Baseline Prompt (Single-Use)")
print("="*70)
print("\nPrompt:")
print(baseline_prompt)
print("\n" + "-"*70)
print("Generating output...\n")

# Generate output using the baseline prompt
baseline_output = llm.invoke(baseline_prompt)

print("Generated Output:")
print("-"*70)
print(baseline_output)
print("-"*70)

### Task 1: Analysis

**What works:**
- Clear, structured output
- Professional tone
- Addresses all required elements

**The Problem:**
- ❌ Hard-coded client name ("DeltaFin")
- ❌ Vague time reference ("this week")
- ❌ Fixed tone and audience
- ❌ Cannot be reused for other projects without complete rewrite
- ❌ Every analyst must write their own version

**Conclusion:** This prompt works for ONE specific situation, but it's not scalable. To make it reusable across the team, we need to identify which parts should become variables.

---
## Task 2: Identify Variable Elements and Convert to a Template

### Teaching Moment: Identifying Variables

Look at the baseline prompt and ask: **What changes between different uses?**

Variables to extract:
1. **Client Name** - DeltaFin, MediTrack, AlphaBank, etc.
2. **Time Period** - Week 4, Sprint 2, Q1, etc.
3. **Progress Items** - What was accomplished
4. **Blockers** - Issues or delays
5. **Next Steps** - Upcoming activities
6. **Tone** - Formal, neutral, friendly
7. **Audience** - Senior stakeholders, internal team, clients

### Creating the Template

In [ ]:
# Task 2: Convert the baseline prompt into a reusable template with variables

# Using Python's string formatting with named placeholders
prompt_template = """Write a concise weekly status summary for the {client_name} project.

Summarize progress made during {time_period}, note {blockers}, and outline {next_steps}
in a {tone} tone suitable for {audience}.

Additional context:
- Progress: {progress}
- Format: {output_format}
"""

print("🟢 TASK 2: Reusable Prompt Template")
print("="*70)
print("\nTemplate with Variables:")
print("-"*70)
print(prompt_template)
print("-"*70)

print("\n✅ Template created successfully!")
print("\nVariables identified:")
variables = ["{client_name}", "{time_period}", "{progress}", "{blockers}",
             "{next_steps}", "{tone}", "{audience}", "{output_format}"]
for var in variables:
    print(f"  • {var}")

### Task 2: Analysis

**Benefits of this approach:**
- ✅ **Reusable**: Works for any project
- ✅ **Consistent**: Same structure for all outputs
- ✅ **Scalable**: Entire team can use it
- ✅ **Flexible**: Adapts to different contexts
- ✅ **Efficient**: No need to rewrite from scratch

**How it works:**
Analysts only need to provide values for the variables. The structure and instructions remain stable, ensuring consistent output quality.

**Conclusion:** You now have a true reusable template. Next, you need to test it with real inputs to ensure it produces reliable outputs.

---
## Task 3: Test, Refine, and Finalize the Prompt Template

### Testing Strategy
We'll test the template with two different projects to verify:
1. Consistent structure across different inputs
2. Proper tone adaptation
3. Audience-appropriate content
4. Complete coverage of requirements

### Test Case 1: DeltaFin Project (Financial Services)

In [ ]:
# Define variables for Project 1: DeltaFin
project1_vars = {
    "client_name": "DeltaFin",
    "time_period": "Week 4",
    "progress": "Completed API integration testing and resolved 15 critical bugs",
    "blockers": "testing delays caused by the external vendor's infrastructure issues",
    "next_steps": "finalize the integration plan and begin UAT preparation",
    "tone": "formal",
    "audience": "senior client stakeholders",
    "output_format": "structured paragraph with clear sections"
}

# Substitute variables into the template
project1_prompt = prompt_template.format(**project1_vars)

print("🟣 TASK 3: Testing Template - Project 1 (DeltaFin)")
print("="*70)
print("\nVariable Values:")
for key, value in project1_vars.items():
    print(f"  {key}: {value}")

print("\n" + "-"*70)
print("Generated Prompt:")
print("-"*70)
print(project1_prompt)

print("\n" + "-"*70)
print("Generating output...\n")

# Generate output
project1_output = llm.invoke(project1_prompt)

print("Generated Status Summary:")
print("-"*70)
print(project1_output)
print("-"*70)

### Test Case 2: MediTrack Project (Healthcare Technology)

In [ ]:
# Define variables for Project 2: MediTrack
project2_vars = {
    "client_name": "MediTrack",
    "time_period": "Sprint 2",
    "progress": "Successfully deployed the patient data dashboard and completed security audit",
    "blockers": "pending approvals from the compliance team for HIPAA certification",
    "next_steps": "complete the data-mapping activity and prepare for pilot testing",
    "tone": "neutral, business-professional",
    "audience": "internal project sponsors",
    "output_format": "bullet-point list with clear categories"
}

# Substitute variables into the template
project2_prompt = prompt_template.format(**project2_vars)

print("🟣 TASK 3: Testing Template - Project 2 (MediTrack)")
print("="*70)
print("\nVariable Values:")
for key, value in project2_vars.items():
    print(f"  {key}: {value}")

print("\n" + "-"*70)
print("Generated Prompt:")
print("-"*70)
print(project2_prompt)

print("\n" + "-"*70)
print("Generating output...\n")

# Generate output
project2_output = llm.invoke(project2_prompt)

print("Generated Status Summary:")
print("-"*70)
print(project2_output)
print("-"*70)

### Task 3: Template Validation

Let's analyze whether our template meets the requirements:

In [ ]:
# Create validation checklist
validation_html = """
<style>
    .validation-box {
        background-color: #f0f8ff;
        border: 2px solid #4CAF50;
        border-radius: 8px;
        padding: 20px;
        margin: 10px 0;
        font-family: Arial, sans-serif;
    }
    .validation-title {
        font-size: 18px;
        font-weight: bold;
        color: #2c3e50;
        margin-bottom: 15px;
    }
    .check-item {
        margin: 8px 0;
        padding-left: 25px;
        position: relative;
    }
    .check-item:before {
        content: "✓";
        position: absolute;
        left: 0;
        color: #4CAF50;
        font-weight: bold;
        font-size: 18px;
    }
</style>
<div class="validation-box">
    <div class="validation-title">✅ Template Validation Results</div>
    <div class="check-item">Generates consistent structure across different projects</div>
    <div class="check-item">Adapts cleanly to different project inputs</div>
    <div class="check-item">Supports tone adjustments (formal vs. neutral)</div>
    <div class="check-item">Adjusts content for different audiences</div>
    <div class="check-item">Eliminates duplicate rewriting across analysts</div>
    <div class="check-item">Maintains professional quality standards</div>
    <div class="check-item">Saves time and ensures consistency</div>
</div>
"""

display(HTML(validation_html))

### Task 3: Conclusion

**Testing Results:**
- ✅ Template produces consistent, high-quality outputs
- ✅ Successfully adapts to different contexts
- ✅ Maintains appropriate tone for each audience
- ✅ Generates properly formatted summaries

**Template Benefits:**
1. **Time Savings**: Analysts don't rewrite prompts from scratch
2. **Consistency**: All outputs follow the same structure
3. **Quality**: Professional standards maintained across team
4. **Scalability**: Works for unlimited number of projects
5. **Flexibility**: Adapts to different tones and audiences

**Conclusion:** Your template now produces clear, consistent outputs for any project in your team. You've successfully created a scalable, reusable prompt template that analysts across the organization can use.

---
## Creating a Template Helper Function

Let's create a reusable function that makes it easy for anyone on your team to use the template.

In [ ]:
def generate_status_summary(client_name, time_period, progress, blockers,
                           next_steps, tone="professional",
                           audience="stakeholders",
                           output_format="structured paragraph"):
    """
    Generate a weekly status summary using the reusable prompt template.

    Parameters:
    - client_name: Name of the client or project
    - time_period: Time frame being reported (e.g., "Week 4", "Sprint 2")
    - progress: What was accomplished
    - blockers: Issues or delays encountered
    - next_steps: Upcoming activities
    - tone: Communication style (default: "professional")
    - audience: Target readers (default: "stakeholders")
    - output_format: Desired format (default: "structured paragraph")

    Returns:
    - Generated status summary as a string
    """

    # Use the template
    prompt = prompt_template.format(
        client_name=client_name,
        time_period=time_period,
        progress=progress,
        blockers=blockers,
        next_steps=next_steps,
        tone=tone,
        audience=audience,
        output_format=output_format
    )

    # Generate output
    result = llm.invoke(prompt)
    return result

print("✅ Template helper function created!")
print("\nUsage example:")
print("""
summary = generate_status_summary(
    client_name="AlphaBank",
    time_period="Q1 2024",
    progress="Completed user authentication module",
    blockers="Waiting for security review",
    next_steps="Deploy to staging environment"
)
""")

---
## Practice Exercise: Use the Template Function

Try creating a status summary for your own project!

In [ ]:
# Practice: Create a status summary for a new project
practice_summary = generate_status_summary(
    client_name="RetailCo",
    time_period="Sprint 5",
    progress="Implemented inventory management system and integrated with POS",
    blockers="Database migration delayed due to legacy system constraints",
    next_steps="complete data migration and conduct user acceptance testing",
    tone="friendly yet professional",
    audience="internal project team and client product owner",
    output_format="bullet points with emoji indicators"
)

print("📝 PRACTICE EXERCISE: RetailCo Project")
print("="*70)
print(practice_summary)
print("="*70)

---
## Bonus: Creating Additional Templates

Now that you understand the concept, let's create templates for other common tasks.

### Template 2: Q&A Response Template for Chatbot

In [ ]:
# Template for Q&A chatbot responses
qa_template = """You are a {role} responding to {query_type} queries.

Context: {context}

Question: {question}

Provide a {response_length} response in a {tone} tone that:
- Directly answers the question
- Is appropriate for {audience}
- {additional_requirements}
"""

print("✅ Q&A Response Template Created")
print("\nTemplate:")
print("-"*70)
print(qa_template)
print("-"*70)

In [ ]:
# Example 1: Customer Support Query
customer_support_vars = {
    "role": "customer support specialist for a banking application",
    "query_type": "account security",
    "context": "Customer is concerned about unauthorized login attempts",
    "question": "How can I secure my account if I notice suspicious activity?",
    "response_length": "concise (3-4 sentences)",
    "tone": "reassuring and helpful",
    "audience": "non-technical banking customers",
    "additional_requirements": "Provides clear action steps and contact information"
}

customer_prompt = qa_template.format(**customer_support_vars)
customer_response = llm.invoke(customer_prompt)

print("🔷 Example 1: Customer Support Response")
print("="*70)
print(f"Question: {customer_support_vars['question']}\n")
print("Response:")
print("-"*70)
print(customer_response)
print("-"*70)

In [ ]:
# Example 2: Internal Employee Query
employee_query_vars = {
    "role": "HR knowledge base assistant",
    "query_type": "company policy",
    "context": "Employee asking about remote work policies",
    "question": "What is the company's policy on hybrid work arrangements?",
    "response_length": "detailed (5-6 sentences)",
    "tone": "informative and professional",
    "audience": "company employees",
    "additional_requirements": "Cites specific policy sections and provides links to full documentation"
}

employee_prompt = qa_template.format(**employee_query_vars)
employee_response = llm.invoke(employee_prompt)

print("🔷 Example 2: Internal HR Query Response")
print("="*70)
print(f"Question: {employee_query_vars['question']}\n")
print("Response:")
print("-"*70)
print(employee_response)
print("-"*70)

### Template 3: Concept Explanation Template

In [ ]:
# Template for explaining concepts to different audiences
explanation_template = """Explain {concept} to {audience}.

Context: {context}

Requirements:
- Use {technical_level} language
- Length: {length}
- Include: {include_elements}
- Tone: {tone}

Provide the explanation:
"""

print("✅ Concept Explanation Template Created")
print("\nTemplate:")
print("-"*70)
print(explanation_template)
print("-"*70)

In [ ]:
# Example: Explain cybersecurity to banking customers
cybersecurity_vars = {
    "concept": "cybersecurity risks in online banking",
    "audience": "banking customers with limited technical knowledge",
    "context": "Recent increase in phishing attacks targeting online banking users",
    "technical_level": "non-technical, everyday",
    "length": "approximately 150 words",
    "include_elements": "real-world examples, practical tips, and what to watch for",
    "tone": "clear, reassuring, and educational"
}

cybersecurity_prompt = explanation_template.format(**cybersecurity_vars)
cybersecurity_explanation = llm.invoke(cybersecurity_prompt)

print("🔷 Example: Cybersecurity Explanation for Banking Customers")
print("="*70)
print(cybersecurity_explanation)
print("-"*70)

In [ ]:
# Example: Explain 5 Why analysis to corporate employees
five_why_vars = {
    "concept": "the 5 Why technique for root cause analysis",
    "audience": "corporate employees and team leads",
    "context": "Training session on problem-solving methodologies",
    "technical_level": "business professional",
    "length": "approximately 200 words",
    "include_elements": "step-by-step process, a workplace example, and when to use it",
    "tone": "instructional and engaging"
}

five_why_prompt = explanation_template.format(**five_why_vars)
five_why_explanation = llm.invoke(five_why_prompt)

print("🔷 Example: 5 Why Analysis Explanation for Corporate Employees")
print("="*70)
print(five_why_explanation)
print("-"*70)

---
## Interactive Template Builder

Use this interactive widget to test the status summary template with your own values.

In [ ]:
# Interactive template builder
def create_interactive_builder():
    # Create input widgets
    client_input = widgets.Text(description='Client:', value='MyProject')
    period_input = widgets.Text(description='Time Period:', value='Week 1')
    progress_input = widgets.Textarea(description='Progress:', value='Completed initial setup')
    blockers_input = widgets.Textarea(description='Blockers:', value='None at this time')
    next_steps_input = widgets.Textarea(description='Next Steps:', value='Begin development')
    tone_dropdown = widgets.Dropdown(
        options=['professional', 'formal', 'casual', 'friendly'],
        description='Tone:',
        value='professional'
    )
    audience_input = widgets.Text(description='Audience:', value='stakeholders')

    generate_button = widgets.Button(description='Generate Summary', button_style='success')
    output_area = widgets.Output()

    def on_generate_click(b):
        with output_area:
            output_area.clear_output()
            print("Generating...\n")
            result = generate_status_summary(
                client_name=client_input.value,
                time_period=period_input.value,
                progress=progress_input.value,
                blockers=blockers_input.value,
                next_steps=next_steps_input.value,
                tone=tone_dropdown.value,
                audience=audience_input.value
            )
            print("Generated Status Summary:")
            print("-" * 70)
            print(result)
            print("-" * 70)

    generate_button.on_click(on_generate_click)

    # Display widgets
    display(widgets.VBox([
        widgets.HTML("<h3>Interactive Template Builder</h3>"),
        client_input, period_input, progress_input, blockers_input,
        next_steps_input, tone_dropdown, audience_input,
        generate_button, output_area
    ]))

# Create the interactive builder
create_interactive_builder()

---
## Summary and Key Takeaways

### What You Learned:

1. **Task 1 - Baseline Prompt**: Created a detailed, single-use prompt to understand requirements

2. **Task 2 - Variable Identification**: Identified changeable elements and converted them to variables

3. **Task 3 - Testing & Refinement**: Validated template with multiple test cases across different contexts

### Benefits of Prompt Templates:

✅ **Time Efficiency**: No need to rewrite prompts from scratch  
✅ **Consistency**: All team members produce uniform outputs  
✅ **Quality**: Professional standards maintained automatically  
✅ **Scalability**: Works for unlimited use cases  
✅ **Flexibility**: Adapts to different contexts and requirements  

### Template Types Created:

1. **Status Summary Template**: Weekly project updates
2. **Q&A Response Template**: Chatbot responses for different contexts
3. **Concept Explanation Template**: Educational content for varied audiences

### Best Practices:

- Start with a working baseline prompt
- Identify truly variable elements (not everything needs to be a variable)
- Test with diverse inputs
- Refine based on results
- Document expected variable formats
- Create helper functions for ease of use

### Technical Stack:
- **Model**: IBM Granite 3.3 8B Instruct
- **Platform**: Replicate
- **Libraries**: langchain_community, replicate, ibm-granite-community, ipywidgets
- **Parameters**: Temperature=0.2, Max Tokens=1024

---
## Your Turn: Create a Custom Template

Now create your own template for a task in your work or studies!

In [ ]:
# Exercise: Create your own custom template

# Step 1: Write a baseline prompt for your use case
my_baseline_prompt = """
# Write your baseline prompt here
# Example: "Draft an email to inform the team about..."
"""

# Step 2: Identify variables and create a template
my_custom_template = """
# Convert your baseline prompt to a template with variables
# Use {variable_name} syntax for placeholders
"""

# Step 3: Test your template
my_test_vars = {
    # Add your variable values here
    # "variable_name": "value"
}

# Uncomment to test:
# my_prompt = my_custom_template.format(**my_test_vars)
# my_output = llm.invoke(my_prompt)
# print(my_output)

print("👉 Complete the sections above to create your custom template!")

---
## Additional Resources

### Template Use Cases to Explore:

1. **Email Templates**: Client communications, internal announcements, follow-ups
2. **Report Templates**: Executive summaries, technical reports, incident reports
3. **Content Templates**: Blog posts, social media, documentation
4. **Analysis Templates**: Data insights, meeting summaries, decision memos
5. **Code Templates**: Documentation, comments, README files

### Next Steps:

- Build a template library for your team
- Create helper functions for frequently used templates
- Share templates across your organization
- Gather feedback and iterate on template design
- Measure time savings and quality improvements

---

**Congratulations!** You've mastered creating reusable prompt templates with IBM Granite models. This skill will save time and improve consistency across your team's AI-generated content.